In [1]:
from imports import *
from catalog_object import *
import mwdust, estimate_PDF
from scipy.interpolate import LinearNDInterpolator as lint
%matplotlib inline


Bad key "xtick.labeltop" on line 358 in
/Users/ryancloutier/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## get pre-GAIA data on all Kepler low mass

Read in data from MAST and cross-match with 2MASS data to get photometric uncertainties.

In [25]:
# get pre-GAIA data on Kepler planet hosts
fname = 'Keplertargets/MAST_all_Keplerlowmassstars.csv'
d = np.loadtxt(fname, delimiter=',').T
KepIDs,ras,decs,Teff,logg,FeH,Rss,Kepmag,Jmag,Hmag,Kmag = d

In [26]:
# reduce numbers
g = Teff <= 4800
KepIDs = KepIDs[g]
ras = ras[g]
decs = decs[g]
Teff = Teff[g]
logg = logg[g]
FeH = FeH[g]
Rss = Rss[g]
Kepmag = Kepmag[g]
Jmag = Jmag[g]
Hmag = Hmag[g]
Kmag = Kmag[g]
N = KepIDs.size
print N

9927


In [28]:
# match to 2MASS sources to get photometric uncertainties
ra2,dec2,_,Jmag2,_,eJmag2,Hmag2,_,eHmag2,Kmag2,_,eKmag2 = np.load('Keplertargets/fp_2mass.fp_psc12298.npy').T
eJmag, eHmag, eKmag = np.zeros(N), np.zeros(N), np.zeros(N)
has2MASS = np.ones(N, dtype=bool)
for i in range(N):
    
    if i % 1e2 == 0:
        print float(i)/N
    
    # search within 10 arcsec
    sep = np.sqrt((ra2 - ras[i])**2 + (dec2 - decs[i])**2)
    close = sep*3600 < 10
    
    # find photometric matches
    jmag, hmag, kmag = Jmag[i], Hmag[i], Kmag[i]
    
    # exact match
    if np.any((Jmag2[close] == jmag) & (Hmag2[close] == hmag) & (Kmag2[close] == kmag)):
        g = (sep*3600 < 10) & (Jmag2 == jmag) & (Hmag2 == hmag) & (Kmag2 == kmag)
        eJmag[i] = eJmag2[g]
        eHmag[i] = eHmag2[g]
        eKmag[i] = eKmag2[g]

    # nearest match
    elif np.any(np.isclose(Jmag2[close],jmag,rtol=.01)) & np.any(np.isclose(Hmag2[close],hmag,rtol=.01)) & np.any(np.isclose(Kmag2[close],kmag,rtol=.01)):
        g = (sep*3600 < 10) & np.isclose(Jmag2,jmag,rtol=.01) & np.isclose(Hmag2,hmag,rtol=.01) & np.isclose(Kmag2,kmag,rtol=.01)
        eJmag[i] = eJmag2[g][0]
        eHmag[i] = eHmag2[g][0]
        eKmag[i] = eKmag2[g][0]
        
    else:
        print i
        has2MASS[i] = False

0.0
0.0100735368188
0.0201470736376
0.0302206104563
0.0402941472751
0.0503676840939
0.0604412209127
0.0705147577314
0.0805882945502
0.090661831369
0.100735368188
0.110808905007
0.120882441825
0.130955978644
0.141029515463
0.151103052282
0.1611765891
0.171250125919
0.181323662738
0.191397199557
0.201470736376
0.211544273194
0.221617810013
0.231691346832
0.241764883651
0.251838420469
0.261911957288
0.271985494107
0.282059030926
0.292132567745
0.302206104563
0.312279641382
0.322353178201
0.33242671502
0.342500251838
0.352573788657
0.362647325476
0.372720862295
0.382794399114
0.392867935932
0.402941472751
0.41301500957
0.423088546389
0.433162083207
0.443235620026
0.453309156845
0.463382693664
0.473456230483
0.483529767301
0.49360330412
0.503676840939
0.513750377758
0.523823914576
0.533897451395
0.543970988214
0.554044525033
0.564118061852
0.57419159867
0.584265135489
0.594338672308
0.604412209127
0.614485745945
0.624559282764
0.634632819583
0.644706356402
0.654779893221
0.664853430039
0.67

In [29]:
# check for missing photometric uncertainties and replace those values
def replace_bad_emag(mags2MASS, emags2MASS, mags, emags, mag_range=.3):
    assert mags.size == emags.size
    print 'There are %i/%i bad photometric uncertainies.'%(np.isnan(emags).sum(), mags.size)
    for i in np.arange(mags.size, dtype=int)[np.isnan(emags)]:
        g = (mags2MASS >= mags[i]-mag_range) & (mags2MASS <= mags[i]+mag_range)
        emags[i] = np.nanmedian(emags2MASS[g])
    return emags
    
eJmag = replace_bad_emag(Jmag2, eJmag2, Jmag, eJmag)
eHmag = replace_bad_emag(Hmag2, eHmag2, Hmag, eHmag)
eKmag = replace_bad_emag(Kmag2, eKmag2, Kmag, eKmag)

There are 25/9927 bad photometric uncertainies.
There are 104/9927 bad photometric uncertainies.
There are 178/9927 bad photometric uncertainies.


## get GAIA data on Kepler low mass stars

Cross-match Kepler low mass stars to GAIA DR2 and save parallaxes, photometry, and positions

In [30]:
# get GAIA data from Megan Bedell's Kepler-GAIA catalog
hdu = fits.open('Keplertargets/kepler_dr2_4arcsec.fits')[1]

hasGAIA = np.zeros(N, dtype=bool)
pars, epars = np.zeros(N), np.zeros(N)
GBPmags, eGBPmags = np.zeros(N), np.zeros(N)
GRPmags, eGRPmags = np.zeros(N), np.zeros(N)
ras, decs = np.zeros(N), np.zeros(N)
ls, bs = np.zeros(N), np.zeros(N)
for i in range(N):
    
    if i % 1e2 == 0:
        print float(i)/N
    
    g = hdu.data['kepid'] == KepIDs[i]
    if g.sum() == 0:   # no match
        pars[i], epars[i] = np.repeat(np.nan, 2)
        GBPmags[i], eGBPmags[i] = np.repeat(np.nan, 2)
        GRPmags[i], eGRPmags[i] = np.repeat(np.nan, 2)
        ras[i], decs[i] = np.repeat(np.nan, 2)
        ls[i], bs[i] = np.repeat(np.nan, 2)
        
    elif g.sum() >= 1:  # at least one match (take the first entry if multiple matches)
        index = 0
        hasGAIA[i] = True
        pars[i], epars[i] = hdu.data['parallax'][g][index], hdu.data['parallax_error'][g][index]

        GBPmags[i] = hdu.data['phot_bp_mean_mag'][g][index]            
        FBP = hdu.data['phot_bp_mean_flux'][g][index]
        eFBP = hdu.data['phot_bp_mean_flux_error'][g][index]
        eGBPmags[i] = -2.5*np.log10(FBP / (FBP+eFBP))
        
        GRPmags[i] = hdu.data['phot_rp_mean_mag'][g][index]            
        FRP = hdu.data['phot_rp_mean_flux'][g][index]
        eFRP = hdu.data['phot_rp_mean_flux_error'][g][index]
        eGRPmags[i] = -2.5*np.log10(FRP / (FRP+eFRP))
        
        ras[i], decs[i] = hdu.data['ra'][g][index], hdu.data['dec'][g][index]
        ls[i], bs[i] = hdu.data['l'][g][index], hdu.data['b'][g][index]

0.0
0.0100735368188
0.0201470736376
0.0302206104563
0.0402941472751
0.0503676840939
0.0604412209127
0.0705147577314
0.0805882945502
0.090661831369
0.100735368188
0.110808905007
0.120882441825
0.130955978644
0.141029515463
0.151103052282
0.1611765891
0.171250125919
0.181323662738
0.191397199557
0.201470736376
0.211544273194
0.221617810013
0.231691346832
0.241764883651
0.251838420469
0.261911957288
0.271985494107
0.282059030926
0.292132567745
0.302206104563
0.312279641382
0.322353178201
0.33242671502
0.342500251838
0.352573788657
0.362647325476
0.372720862295
0.382794399114
0.392867935932
0.402941472751
0.41301500957
0.423088546389
0.433162083207
0.443235620026
0.453309156845
0.463382693664
0.473456230483
0.483529767301
0.49360330412
0.503676840939
0.513750377758
0.523823914576
0.533897451395
0.543970988214
0.554044525033
0.564118061852
0.57419159867
0.584265135489
0.594338672308
0.604412209127
0.614485745945
0.624559282764
0.634632819583
0.644706356402
0.654779893221
0.664853430039
0.67

In [33]:
# flag negative parallaxes
g = pars <= 0
hasGAIA[g] = False

/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in less_equal
  from ipykernel import kernelapp as app


In [34]:
# save progress
self = catalog_object('Results/Keplerlowmassstars_all_GAIA_v1')
self.Nstars = N
self.hasGAIA = hasGAIA
self.pars, self.epars = pars, epars
self.GBPmags, self.eGBPmags = GBPmags, eGBPmags
self.GRPmags, self.eGRPmags = GRPmags, eGRPmags
self.ras, self.decs = ras, decs
self.ls, self.bs = ls, bs
self._pickleobject()

## compute distance posteriors (from Bailor-Jones)

In [31]:
# get distance posteriors from Bailor-Jones R scripts
cwd = os.getcwd()
os.chdir('GaiaDistances/')
prefix = 'KepID'
overwrite = False
for i in range(N):
       
    cmd_prefix = 'Rscript get_dist_post.R %s'%prefix
    cmd = '%s_%i %.6e %.6e %.6f %.6f'%(cmd_prefix, KepIDs[i], pars[i], epars[i], ls[i], bs[i])

    # if posterior doesn't exist, compute it using the R script
    fout = 'DistancePosteriors/%s_%i.csv'%(prefix, KepIDs[i])
    if ((not os.path.exists(fout)) or overwrite) and (hasGAIA[i]):
        os.system(cmd)

os.chdir(cwd)

In [32]:
# save distance point estimates
Nsamp = 1000
dists, ehidists, elodists = np.zeros(N), np.zeros(N), np.zeros(N)
mus, ehimus, elomus = np.zeros(N), np.zeros(N), np.zeros(N)
for i in range(N):
    
    if hasGAIA[i]:
        # get distance posterior
        fname = 'GaiaDistances/DistancePosteriors/%s_%i.csv'%(prefix, KepIDs[i])
        try:  
            distarr, probarr = np.loadtxt(fname, delimiter=',', skiprows=1, usecols=(1,2)).T
            samp_dist = np.random.choice(distarr, Nsamp, p=probarr/probarr.sum()) + np.random.randn(Nsamp)*1e-2

            # save distance point estimates
            v = np.percentile(samp_dist, (16,50,84))
            dists[i], ehidists[i], elodists[i] = v[1], v[2]-v[1], v[1]-v[0]
    
            # sample and save distance modulus
            samp_mu = 5*np.log10(samp_dist) - 5
            v = np.percentile(samp_mu, (16,50,84))
            mus[i], ehimus[i], elomus[i] = v[1], v[2]-v[1], v[1]-v[0]
        
        except IOError:
            dists[i], ehidists[i], elodists[i] = np.repeat(np.nan, 3)
            mus[i], ehimus[i], elomus[i] = np.repeat(np.nan, 3)
            hasGAIA[i] = False
    else:
        dists[i], ehidists[i], elodists[i] = np.repeat(np.nan, 3)
        mus[i], ehimus[i], elomus[i] = np.repeat(np.nan, 3)

In [35]:
# save progress
self.dists, self.ehidists, self.elodists = dists, ehidists, elodists
self.mus, self.ehimus, self.elomus = mus, ehimus, elomus
self._pickleobject()

## estimate extinction using mwdust

In [37]:
def _compute_AK_mwdust(ls, bs, dist, edist, eAK_frac=.3, RV=3.1):
    '''Using the EB-V map from 2014MNRAS.443.2907S and the extinction vector
    RK = 0.31 from Schlafly and Finkbeiner 2011 (ApJ 737, 103). 
    Where does RV=3.1 come from? See the MIST BC tables where RV is fixed to 3.1'''
    dustmapK = mwdust.Combined15(filter='2MASS Ks')
    dustmapV = mwdust.Combined15(filter=None)  # returns E(B-V) rather than A_lambda
    dist_kpc, edist_kpc = np.ascontiguousarray(dist)*1e-3, np.ascontiguousarray(edist)*1e-3
    ls, bs = np.ascontiguousarray(ls), np.ascontiguousarray(bs)
    AK, eAK = np.zeros(ls.size), np.zeros(ls.size)
    AV = np.zeros(ls.size)
    for i in range(ls.size):
        v = dustmapK(ls[i], bs[i],
                     np.array([dist_kpc[i], dist_kpc[i]+edist_kpc[i]]))
        AK[i], eAK[i] = v[0], np.sqrt(abs(np.diff(v))**2 + (eAK_frac*v[0])**2)
        v = dustmapV(ls[i], bs[i],
                     np.array([dist_kpc[i], dist_kpc[i]+edist_kpc[i]]))
        AV[i] = v[0]*RV
    return AK, eAK, AV


def MAD(arr):
    return np.median(abs(arr - np.median(arr)))
    

AKs, eAKs, AVs = np.zeros(N), np.zeros(N), np.zeros(N)
for i in range(N):
    
    if i % 1e2 == 0:
        print float(i)/N
    
    if hasGAIA[i]:

        # get distance posterior
        fname = 'GaiaDistances/DistancePosteriors/KepID_%i.csv'%KepIDs[i]
        distarr, probarr = np.loadtxt(fname, delimiter=',', skiprows=1, usecols=(1,2)).T
        samp_dist = np.random.choice(distarr, Nsamp, p=probarr/probarr.sum()) + np.random.randn(Nsamp)*1e-2
        
        # compute K-band extinction
        AKs[i], eAKs[i], AVs[i] = _compute_AK_mwdust(ls[i], bs[i], dists[i], MAD(samp_dist))

0.0
0.0100735368188
0.0201470736376
0.0302206104563
0.0402941472751
0.0503676840939
0.0604412209127
0.0705147577314
0.0805882945502
0.090661831369
0.100735368188
0.110808905007
0.120882441825
0.130955978644
0.141029515463
0.151103052282
0.1611765891
0.171250125919
0.181323662738
0.191397199557
0.201470736376
0.211544273194
0.221617810013
0.231691346832
0.241764883651
0.251838420469
0.261911957288
0.271985494107
0.282059030926
0.292132567745
0.302206104563
0.312279641382
0.322353178201
0.33242671502
0.342500251838
0.352573788657
0.362647325476
0.372720862295
0.382794399114
0.392867935932
0.402941472751
0.41301500957
0.423088546389
0.433162083207
0.443235620026
0.453309156845
0.463382693664
0.473456230483
0.483529767301
0.49360330412
0.503676840939
0.513750377758
0.523823914576
0.533897451395
0.543970988214
0.554044525033
0.564118061852
0.57419159867
0.584265135489
0.594338672308
0.604412209127
0.614485745945
0.624559282764
0.634632819583
0.644706356402
0.654779893221
0.664853430039
0.67

In [38]:
# save progress
self.AKs, self.eAKs, self.AVs = AKs, eAKs, AVs
self._pickleobject()

## setup functions to compute bolometric corrections

In [39]:
def _interpolate_BCK(Teff, e_Teff, logg, FeH, Av):
    '''Interpolate the MIST bolometric correction grids to get a star's K-band bolometric correction.'''
    # select a metallicity grid
    FeHgrid = np.array([-4,-3.5,-3,-2.75,-2.5,-2.25,-2,-1.75,-1.5,-1.25,-1,-.75,-.5,-.25,0,.25,.5,.75])
    inds, coeffs = _get_interpolation_coeffs(FeHgrid, FeH)
    if np.any(coeffs == 0):
        FeHs = FeHgrid[inds[coeffs>0]]
        coeffs = np.ones(1)
    else:
        FeHs = FeHgrid[inds]
    
    # interpolate over remaining parameters for all metallicity grids and with sampled Teff
    BCKs = np.zeros((FeHs.size, 2))
    for i in range(FeHs.size):
        
        # get BC grid for a fixed Fe/H
        label = 'p' if FeHs[i] >= 0 else 'm'
        fname = 'UBVRIplus/feh%s%.3d.UBVRIplus'%(label, abs(FeHs[i])*1e2)
        Teffgrid, logggrid, Avgrid, BCKgrid  = np.loadtxt(fname, skiprows=5, usecols=(0,1,3,12)).T
        
        # compute a separate BCK for Teff across its 1 sigma uncertainty to estimate the uncertainty in BCK
        # because it is dominated by uncertainies in Teff which have the strongest effect on the stellar SED
        BCKs_Teff, j = np.zeros(0), 0
        for t in Teff+np.arange(2)*e_Teff:
            # interpolate to get BCK
            lint_BCK = lint(np.array([Teffgrid,logggrid,Avgrid]).T, BCKgrid)
            BCKs[i,j] = np.append(BCKs_Teff, float(lint_BCK(t, logg, Av)) * coeffs[i])
            j += 1
            
    # sum contributions from each Fe/H
    BCKs = np.sum(BCKs, axis=0)
    assert BCKs.size == 2
    BCK, e_BCK = BCKs[0], abs(float(np.diff(BCKs)))
    return BCK, e_BCK

                     
def _get_interpolation_coeffs(arr, val):
    if val < arr.min():
        return np.argsort(arr)[:1], np.ones(1)
    elif val > arr.max():
        return np.argsort(arr)[-1:], np.ones(1)
    else:
        edgeinds = np.argsort(abs(arr-val))[:2]
        edgevals = arr[edgeinds]
        diff = abs(float(np.diff(edgevals)))
        c1 = abs(val-edgevals[0]) / diff
        coeffs = np.array([1-c1, c1]).reshape(2)
        return edgeinds, coeffs

## compute new stellar parameters

In [40]:
def _sample_Rs_from_MK_Mdwarfs(samp_MK):
    '''Use relation from Mann+2015 (table 1) for M dwarfs'''
    if (np.median(samp_MK) > 4.6) & (np.median(samp_MK) < 9.8):
        a, b, c, Rs_sigma_frac = 1.9515, -.3520, .01680, .0289
        p = np.poly1d((c,b,a))
        samp_MK_tmp = np.copy(samp_MK)
        #samp_MK_tmp[(samp_MK<=4.6) | (samp_MK>=9.8)] = np.nan
        samp_Rs = p(samp_MK_tmp)
        samp_Rs += np.random.normal(0, samp_Rs*Rs_sigma_frac, samp_MK.size)
    else:
        samp_Rs = np.zeros_like(samp_MK) + np.nan
    return samp_Rs


def _sample_Rs_from_MK_Kdwarfs(samp_MK, samp_BCK, samp_Teff):
    if (np.median(samp_MK) < 4.6):
        samp_Mbol = samp_MK + samp_BCK
        samp_L = 3.0128e28 * 10**(-.4*samp_Mbol)  # Watts
        sigma = 5.67e-8
        samp_Rs = rvs.m2Rsun(np.sqrt(samp_L / (4*np.pi*sigma*samp_Teff**4)))
    else:
        samp_Rs = np.zeros_like(samp_MK) + np.nan
    return samp_Rs
        

def _sample_Teff_from_colors(samp_GBPmag, samp_GRPmag, samp_Jmag, samp_Hmag,
                             Teff_scatter=49):
    '''Use the relation from Mann+2015 (table 2)'''
    a, b, c, d, e, f, g = 3.172, -2.475, 1.082, -.2231, .01738, .08776, -.04355
    pG = np.poly1d((e,d,c,b,a))
    p2 = np.poly1d((g,f,0))
    samp_Teff = 35e2 * (pG(samp_GBPmag-samp_GRPmag) + p2(samp_Jmag-samp_Hmag)) \
                + np.random.normal(0, Teff_scatter, samp_Jmag.size)
    return samp_Teff


def _sample_Ms_from_MK_Mdwarfs(samp_MK):
    '''Use relation from Benedict+2016'''
    if (np.median(samp_MK) >= 5) & (np.median(samp_MK) < 10):
        c0 = np.random.normal(.2311, 4e-4, samp_MK.size)
        c1 = np.random.normal(-.1352, 7e-4, samp_MK.size)
        c2 = np.random.normal(.04, 5e-4, samp_MK.size)
        c3 = np.random.normal(.0038, 2e-4, samp_MK.size)
        c4 = np.random.normal(-.0032, 1e-4, samp_MK.size)
        samp_MK_tmp = np.copy(samp_MK)
        #samp_MK_tmp[(samp_MK<=4.6) | (samp_MK>10)] = np.nan
        #samp_MK_tmp[samp_MK>=10] = np.nan
        dMK = samp_MK_tmp - 7.5
        samp_Ms = c0 + c1*dMK + c2*dMK**2 + c3*dMK**3 + c4*dMK**4
    else:
        samp_Ms = np.zeros_like(samp_MK) + np.nan
    return samp_Ms


def _sample_Ms_from_MK_Kdwarfs(samp_MK):
    '''Empirical relation with stellar mass from Mann+2018 (Table 6)'''
    if (np.median(samp_MK) >= 4) & (np.median(samp_MK) < 11):
        zp = 7.5
        ais = np.array([-.642, -.208, -8.43e-4, 7.87e-3, 1.42e-4, -2.13e-4])
        samp_Ms = np.array([10**np.sum(ais*(mk - zp)**np.arange(ais.size)) for mk in samp_MK])
        sig_Ms = _get_sigMs(np.median(samp_MK))
        samp_Ms += np.random.normal(0, sig_Ms, samp_MK.size)
    else:
        samp_Ms = np.zeros_like(samp_MK) + np.nan
    return samp_Ms
        

def _get_sigMs(MK):
    '''Return the characeristic scatter in stellar mass given MK 
    from the empirial relation in Mann+2018 (Table 7)'''
    assert (MK >= 4) & (MK <= 11)
    MKarr = np.delete(np.arange(4,11.1,.5), 7)
    sig_Mss = np.array([28,16,15,12,9.9,8.1,6.4,3.9,3.2,2.6,2.3,2.2,2.1,2.4]) * 1e-3
    assert MKarr.size == sig_Mss.size
    s = np.argsort(abs(MKarr - MK))
    return sig_Mss[s][0]
    
    
def _compute_logg(Ms, Rs):
    G = 6.67e-11
    return np.log10(G*rvs.Msun2kg(Ms)*1e2 / rvs.Rsun2m(Rs)**2)


def _compute_percentiles(samples):
    v = np.percentile(samples, (16,50,84))
    return v[1], v[2]-v[1], v[1]-v[0]

In [41]:
# initialize arrays
nanarr = np.repeat(np.nan, N)
isMdwarf = np.zeros(N, dtype=bool)
BCKs, eBCKs = np.copy(nanarr), np.copy(nanarr)
MKs, ehiMKs, eloMKs = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
Rss2, ehiRss2, eloRss2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
Teffs2, ehiTeffs2, eloTeffs2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
Mss2, ehiMss2, eloMss2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
loggs2, ehiloggs2, elologgs2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)

In [ ]:
# compute stellar parameters
for i in range(N):
    
    print i, float(i)/N
    
    if hasGAIA[i]:
        # sample input parameters
        samp_Jmag = np.random.randn(Nsamp)*eJmag[i] + Jmag[i]
        samp_Hmag = np.random.randn(Nsamp)*eHmag[i] + Hmag[i]
        samp_Kmag = np.random.randn(Nsamp)*eKmag[i] + Kmag[i]
        samp_GBPmag = np.random.randn(Nsamp)*eGBPmags[i] + GBPmags[i]
        samp_GRPmag = np.random.randn(Nsamp)*eGRPmags[i] + GRPmags[i]
        samp_AK = np.random.randn(Nsamp)*eAKs[i] + AKs[i]
        _,_,samp_mu = estimate_PDF.get_samples_from_percentiles(mus[i], ehimus[i], elomus[i], Nsamp=Nsamp)
        #_,_,samp_logg = estimate_PDF.get_samples_from_percentiles(logg[i], ehilogg[i], abs(elologg[i]), Nsamp=Nsamp)
        #_,_,samp_FeH = estimate_PDF.get_samples_from_percentiles(FeH[i], ehiFeH[i], abs(eloFeH[i]), Nsamp=Nsamp)
        
        # compute new effective temperature
        samp_Teff = _sample_Teff_from_colors(samp_GBPmag, samp_GRPmag, samp_Jmag, samp_Hmag)
        Teffs2[i], ehiTeffs2[i], eloTeffs2[i] = _compute_percentiles(samp_Teff)
        if Teffs2[i] - eloTeffs2[i] > 4700:
            pass
        
        # compute MK to see if we have an M dwarf
        samp_MK = samp_Kmag - samp_mu - samp_AK
        MKs[i], ehiMKs[i], eloMKs[i] = _compute_percentiles(samp_MK)
        isMdwarf[i] = (MKs[i] > 4.6) & (MKs[i] < 9.8)
        
        # compute bolometric corrections for non-M dwarfs and update stellar radii
        if not isMdwarf[i]:
            BCKs[i], eBCKs[i] = _interpolate_BCK(Teffs2[i], ehiTeffs2[i], logg[i], FeH[i], AVs[i])
            samp_BCK = np.random.randn(Nsamp)*eBCKs[i] + BCKs[i]
            samp_Rs = _sample_Rs_from_MK_Kdwarfs(samp_MK, samp_BCK, samp_Teff)
        # M dwarf radii
        else:
            samp_Rs = _sample_Rs_from_MK_Mdwarfs(samp_MK)

        # compute masses from empirical relations depending on MK
        if (MKs[i] >= 5):
            samp_Ms = _sample_Ms_from_MK_Mdwarfs(samp_MK)  # Benedict+2106
        else:
            samp_Ms = _sample_Ms_from_MK_Kdwarfs(samp_MK)
            
        # update stellar parameters
        Rss2[i], ehiRss2[i], eloRss2[i] = _compute_percentiles(samp_Rs)
        Mss2[i], ehiMss2[i], eloMss2[i] = _compute_percentiles(samp_Ms)
        loggs2[i], ehiloggs2[i], elologgs2[i] = _compute_percentiles(_compute_logg(samp_Ms, samp_Rs))

0 0.0


/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


1 0.000100735368188
2 0.000201470736376
3 0.000302206104563
4 0.000402941472751
5 0.000503676840939


/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:80: RuntimeWarning: invalid value encountered in log10


6 0.000604412209127
7 0.000705147577314
8 0.000805882945502
9 0.00090661831369
10 0.00100735368188
11 0.00110808905007
12 0.00120882441825
13 0.00130955978644
14 0.00141029515463
15 0.00151103052282
16 0.001611765891
17 0.00171250125919
18 0.00181323662738
19 0.00191397199557
20 0.00201470736376
21 0.00211544273194
22 0.00221617810013
23 0.00231691346832
24 0.00241764883651
25 0.00251838420469
26 0.00261911957288
27 0.00271985494107
28 0.00282059030926
29 0.00292132567745
30 0.00302206104563
31 0.00312279641382
32 0.00322353178201
33 0.0033242671502
34 0.00342500251838
35 0.00352573788657
36 0.00362647325476
37 0.00372720862295
38 0.00382794399114
39 0.00392867935932
40 0.00402941472751
41 0.0041301500957
42 0.00423088546389
43 0.00433162083207
44 0.00443235620026
45 0.00453309156845
46 0.00463382693664
47 0.00473456230483
48 0.00483529767301
49 0.0049360330412
50 0.00503676840939
51 0.00513750377758
52 0.00523823914576
53 0.00533897451395
54 0.00543970988214
55 0.00554044525033
56 0.0

419 0.0422081192707
420 0.0423088546389
421 0.0424095900071
422 0.0425103253752
423 0.0426110607434
424 0.0427117961116
425 0.0428125314798
426 0.042913266848
427 0.0430140022162
428 0.0431147375844
429 0.0432154729526
430 0.0433162083207
431 0.0434169436889
432 0.0435176790571
433 0.0436184144253
434 0.0437191497935
435 0.0438198851617
436 0.0439206205299
437 0.0440213558981
438 0.0441220912662
439 0.0442228266344
440 0.0443235620026
441 0.0444242973708
442 0.044525032739
443 0.0446257681072
444 0.0447265034754
445 0.0448272388436
446 0.0449279742117
447 0.0450287095799
448 0.0451294449481
449 0.0452301803163
450 0.0453309156845
451 0.0454316510527
452 0.0455323864209
453 0.0456331217891
454 0.0457338571572
455 0.0458345925254
456 0.0459353278936
457 0.0460360632618
458 0.04613679863
459 0.0462375339982
460 0.0463382693664
461 0.0464390047346
462 0.0465397401028
463 0.0466404754709
464 0.0467412108391
465 0.0468419462073
466 0.0469426815755
467 0.0470434169437
468 0.0471441523119
469 

832 0.0838118263322
833 0.0839125617004
834 0.0840132970686
835 0.0841140324368
836 0.084214767805
837 0.0843155031732
838 0.0844162385414
839 0.0845169739095
840 0.0846177092777
841 0.0847184446459
842 0.0848191800141
843 0.0849199153823
844 0.0850206507505
845 0.0851213861187
846 0.0852221214869
847 0.085322856855
848 0.0854235922232
849 0.0855243275914
850 0.0856250629596
851 0.0857257983278
852 0.085826533696
853 0.0859272690642
854 0.0860280044324
855 0.0861287398005
856 0.0862294751687
857 0.0863302105369
858 0.0864309459051
859 0.0865316812733
860 0.0866324166415
861 0.0867331520097
862 0.0868338873779
863 0.086934622746
864 0.0870353581142
865 0.0871360934824
866 0.0872368288506
867 0.0873375642188
868 0.087438299587
869 0.0875390349552
870 0.0876397703234
871 0.0877405056915
872 0.0878412410597
873 0.0879419764279
874 0.0880427117961
875 0.0881434471643
876 0.0882441825325
877 0.0883449179007
878 0.0884456532689
879 0.0885463886371
880 0.0886471240052
881 0.0887478593734
882 0

1244 0.125314798026
1245 0.125415533394
1246 0.125516268762
1247 0.12561700413
1248 0.125717739498
1249 0.125818474867
1250 0.125919210235
1251 0.126019945603
1252 0.126120680971
1253 0.126221416339
1254 0.126322151707
1255 0.126422887076
1256 0.126523622444
1257 0.126624357812
1258 0.12672509318
1259 0.126825828548
1260 0.126926563917
1261 0.127027299285
1262 0.127128034653
1263 0.127228770021
1264 0.127329505389
1265 0.127430240758
1266 0.127530976126
1267 0.127631711494
1268 0.127732446862
1269 0.12783318223
1270 0.127933917598
1271 0.128034652967
1272 0.128135388335
1273 0.128236123703
1274 0.128336859071
1275 0.128437594439
1276 0.128538329808
1277 0.128639065176
1278 0.128739800544
1279 0.128840535912
1280 0.12894127128
1281 0.129042006649
1282 0.129142742017
1283 0.129243477385
1284 0.129344212753
1285 0.129444948121
1286 0.129545683489
1287 0.129646418858
1288 0.129747154226
1289 0.129847889594
1290 0.129948624962
1291 0.13004936033
1292 0.130150095699
1293 0.130250831067
1294 

1658 0.167019240455
1659 0.167119975824
1660 0.167220711192
1661 0.16732144656
1662 0.167422181928
1663 0.167522917296
1664 0.167623652664
1665 0.167724388033
1666 0.167825123401
1667 0.167925858769
1668 0.168026594137
1669 0.168127329505
1670 0.168228064874
1671 0.168328800242
1672 0.16842953561
1673 0.168530270978
1674 0.168631006346
1675 0.168731741715
1676 0.168832477083
1677 0.168933212451
1678 0.169033947819
1679 0.169134683187
1680 0.169235418555
1681 0.169336153924
1682 0.169436889292
1683 0.16953762466
1684 0.169638360028
1685 0.169739095396
1686 0.169839830765
1687 0.169940566133
1688 0.170041301501
1689 0.170142036869
1690 0.170242772237
1691 0.170343507606
1692 0.170444242974
1693 0.170544978342
1694 0.17064571371
1695 0.170746449078
1696 0.170847184446
1697 0.170947919815
1698 0.171048655183
1699 0.171149390551
1700 0.171250125919
1701 0.171350861287
1702 0.171451596656
1703 0.171552332024
1704 0.171653067392
1705 0.17175380276
1706 0.171854538128
1707 0.171955273497
1708 

## save parameters to a catalog object

In [14]:
# flag stars of interest for occurrence rate calculations
iscool = np.zeros(N, dtype=bool)
g = (Teffs2 - eloTeffs2 <= 4700) & (loggs2 + ehiloggs2 > 4) & (Rss2 - eloRss2 < .75)
iscool[g] = True

/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in less_equal
  app.launch_new_instance()
/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater
  app.launch_new_instance()
/Users/ryancloutier/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


In [16]:
self = catalog_object('Results/Keplerlowmassstars_withplanets_GAIA_v1')
self.Nstars = N
self.hasGAIA, self.isMdwarf, self.iscool = hasGAIA, isMdwarf, iscool
self.N_hasGAIA, self.N_isMdwarf, self.N_iscool = self.hasGAIA.sum(), self.isMdwarf.sum(), self.iscool.sum()

# positions
self.ras, self.decs = ras, decs
self.ls, self.bs = ls, bs

# photometry
self.GBPmags, self.e_GBPmags = GBPmags, eGBPmags
self.GRPmags, self.e_GRPmags = GRPmags, eGRPmags
self.Jmags, self.e_Jmags = Jmag, eJmag
self.Hmags, self.e_Hmags = Hmag, eHmag
self.Kmags, self.e_Kmags = Kmag, eKmag

# GAIA distances
self.dists, self.ehi_dists, self.elo_dists = dists, ehidists, elodists
self.mus, self.ehi_mus, self.elo_mus = mus, ehimus, elomus
self.MKs, self.ehi_MKs, self.elo_MKs = MKs, ehiMKs, eloMKs
self.AKs, self.e_AKs, self.AVs = AKs, eAKs, AVs

# updated stellar parameters
self.BCKs, self.e_BCKs = BCKs, eBCKs
self.Teffs2, self.ehi_Teffs2, self.elo_Teffs2 = Teffs2, ehiTeffs2, eloTeffs2
self.Rss2, self.ehi_Rss2, self.elo_Rss2 = Rss2, ehiRss2, eloRss2
self.Mss2, self.ehi_Mss2, self.elo_Mss2 = Mss2, ehiMss2, eloMss2
self.loggs2, self.ehi_loggs2, self.elo_loggs2 = loggs2, ehiloggs2, elologgs2

# add previous stellar parameters
self.Teffs1, self.ehi_Teffs1, self.elo_Teffs1 = Teff, ehiTeff, abs(eloTeff)
self.loggs1, self.ehi_loggs1, self.elo_loggs1 = logg, ehilogg, abs(elologg)
self.FeH1, self.ehi_FeH1, self.elo_FeH1 = FeH, ehiFeH, abs(eloFeH)
self.Rss1, self.ehi_Rss1, self.elo_Rss1 = Rss, ehiRs, abs(eloRs)
self.Mss1, self.ehi_Mss1, self.elo_Mss1 = Mss, ehiMs, abs(eloMs)

# planet parameters
self.Ps, self.ehi_Ps, self.elo_Ps = Ps, ehiP, abs(eloP)
self.rpRs, self.ehi_rpRs, self.elo_rpRs = rpRs, ehirpRs, abs(elorpRs)
self.rps1, self.ehi_rps1, self.elo_rps1 = rps, ehirp, abs(elorp)

# update planet radii
self.rps2, self.ehi_rps2, self.elo_rps2 = np.zeros(N), np.zeros(N), np.zeros(N)
for i in range(N):
    _,_,samp_rpRs = estimate_PDF.get_samples_from_percentiles(self.rpRs[i], self.ehi_rpRs[i], self.elo_rpRs[i], Nsamp=Nsamp)
    _,_,samp_Rs = estimate_PDF.get_samples_from_percentiles(self.Rss2[i], self.ehi_Rss2[i], self.elo_Rss2[i], Nsamp=Nsamp)
    samp_rp = rvs.m2Rearth(rvs.Rsun2m(samp_rpRs*samp_Rs))
    self.rps2[i], self.ehi_rps2[i], self.elo_rps2[i] = _compute_percentiles(samp_rp)
    
self._pickleobject()